In [1]:
import io
import re

In [69]:
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import num2words
import gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import Text8Corpus
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [70]:
def train_test_split(idx, label, test_size=0.15):
    from sklearn.model_selection import StratifiedShuffleSplit
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=0)
    for train_index, test_index in sss.split(idx, label):
        df_train = df.iloc[train_index]
        df_test = df.iloc[test_index]
    return df_train, df_test

In [71]:
df = pd.read_csv('data/train.txt', sep='\t')

In [72]:
df.head()

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [73]:
idx, label = df.id.values, df.label.values

In [74]:
df_train, df_test = train_test_split(idx, label)

In [75]:
df_train.head()

,id,turn1,turn2,turn3,label
28305,28305,Please your nambar,Replied. My bad.,Please call me,others
13980,13980,I did ask,now you did,Tell ms,others
25230,25230,YOU CHANGE THE TOPIC,what if i dont change the topic?,THEN I DON&apos;T TLAK TO YOU,angry
4941,4941,You’re never learning,I've only had my first lesson this week but I ...,You are dumb,angry
5744,5744,Lol but I don't,same,😂😂😂,happy


In [76]:
df_test.head()

,id,turn1,turn2,turn3,label
16917,16917,That is fantastic place,I shall visit someday :),Hmm,others
9876,9876,I really need a gf,You will when you are old and alone.,☺️☺️,happy
28636,28636,Yeah,Where now?,"Now , at MY bed",others
1962,1962,That's interesting,I've been waiting for this one.,Tell me something more about you?,others
4165,4165,Which means,Confused is correct!,U wanna say something ??,others


In [77]:
# df_test.to_csv('data/test_split.csv', sep='\t', index=None, header=True)

In [78]:
# df_train.to_csv('data/train_split.csv', sep='\t', index=None, header=True)

In [82]:
label2emotion = {0:"others", 1:"happy", 2: "sad", 3:"angry"}
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}
def preprocessData(dataFilePath, mode):
    """Load data from a file, process and return indices, conversations and labels in separate lists
    Input:
        dataFilePath : Path to train/test file to be processed
        mode : "train" mode returns labels. "test" mode doesn't return labels.
    Output:
        indices : Unique conversation ID list
        conversations : List of 3 turn conversations, processed and each turn separated by the <eos> tag
        labels : [Only available in "train" mode] List of labels
    """
    pos_emoticons=["(^.^)","(^-^)","(^_^)","(^_~)","(^3^)","(^o^)","(~_^)","*)",":)",":*",":-*",":]",":^)",":}",
                   ":>",":3",":b",":-b",":c)",":D",":-D",":O",":-O",":o)",":p",":-p",":P",":-P",":Þ",":-Þ",":X",
                   ":-X",";)",";-)",";]",";D","^)","^.~","_)m"," ~.^","<=8","<3","<333","=)","=///=","=]","=^_^=",
                   "=<_<=","=>.<="," =>.>="," =3","=D","=p","0-0","0w0","8D","8O","B)","C:","d'-'","d(>w<)b",":-)",
                   "d^_^b","qB-)","X3","xD","XD","XP","ʘ‿ʘ","❤","💜","💚","💕","💙","💛","💓","💝","💖","💞",
                   "💘","💗","😗","😘","😙","😚","😻","😀","😁","😃","☺","😄","😆","😇","😉","😊","😋","😍",
                   "😎","😏","😛","😜","😝","😮","😸","😹","😺","😻","😼","👍", "😂", "🙂", "😽", "🤣",
                  "👌", "🌞", ":')"]

    neg_emoticons=["--!--","(,_,)","(-.-)","(._.)","(;.;)9","(>.<)","(>_<)","(>_>)","(¬_¬)","(X_X)",":&",":(",":'(",
                   ":-(",":-/",":-@[1]",":[",":\\",":{",":<",":-9",":c",":S",";(",";*(",";_;","^>_>^","^o)","_|_",
                   "`_´","</3","<=3","=/","=\\",">:(",">:-(","💔","☹️","😌","😒","😓","😔","😕","😖","😞","😟",
                   "😠","😡","😢","😣","😤","😥","😦","😧","😨","😩","😪","😫","😬","😭","😯","😰","😱","😲",
                   "😳","😴","😷","😾","😿","🙀","💀","👎", "👿", "☹", "🙁", "🤢",
                  "✋"]
    neutral_emoticons = ["😶", "🙄", "😅", "🤐", "😑", "😐"]

    # Emails
    emailsRegex=re.compile(r'[\w\.-]+@[\w\.-]+')

    # Mentions
    userMentionsRegex=re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')

    #Urls
    urlsRegex=re.compile('r(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+') # It may not be handling all the cases like t.co without http

    #Numerics
    numsRegex=re.compile(r"\b\d+\b")

    punctuationNotEmoticonsRegex=re.compile(r'([!?.]){2,}')
    
    elongatedWords = re.compile(r'\b(\S*?)(.)\2{2,}\b')
    allCaps = re.compile(r"((?![<]*}) [A-Z][A-Z]+)")

    emoticonsDict = {}
    for i,each in enumerate(pos_emoticons):
        emoticonsDict[each]= ' <SMILE> '
    for i,each in enumerate(neg_emoticons):
        emoticonsDict[each]=' <SADFACE> '
    for i,each in enumerate(neutral_emoticons):
        emoticonsDict[each]=' <NEUTRALFACE> '
    # use these three lines to do the replacement
    rep = dict((re.escape(k), v) for k, v in emoticonsDict.items())
    emoticonsPattern = re.compile("|".join(rep.keys()))
    indices = []
    conversations = []
    labels = []
    u1 = []
    u2 = []
    u3 = []
    indices = []
    
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for row in finput:
            # Convert multiple instances of . ? ! , to single instance
            # okay...sure -> okay . sure
            # okay???sure -> okay ? sure
            # Add whitespace around such punctuation
            # okay!sure -> okay ! sure
#             repeatedChars = ['.', '?', '!', ',']
            
            items = row.strip('\n').split('\t')
            line = '\t'.join(items[1:4])
            line = emoticonsPattern.sub(lambda m: rep[re.escape(m.group(0))], line.strip())
            line = userMentionsRegex.sub(' <USER> ', line )
            line = emailsRegex.sub(' <EMAIL> ', line )
            line=urlsRegex.sub(' <URL> ', line)
            line=numsRegex.sub(' <NUMBER> ',line)
            line=punctuationNotEmoticonsRegex.sub(r' \1 <REPEAT> ',line)
            line = elongatedWords.sub(r'\1\2 <ELONG> ', line)
            line = allCaps.sub(r'\1 <ALLCAPS> ', line)
#             line=re.sub(r'(.)\1{2,}', r'\1\1',line)
            line = line.strip().split('\t')
            line_0 = ' '.join([stemmer.stem(x) for x in line[0].split()])
            line_1 = ' '.join([stemmer.stem(x) for x in line[1].split()])
            line_2 = ' '.join([stemmer.stem(x) for x in line[2].split()])
            if mode == "train":
                # Train data contains id, 3 turns and label
                label = emotion2label[items[4]]
                labels.append(label)
            
            conv = ' '.join(line)
            
            u1.append(line_0)
            u2.append(line_1)
            u3.append(line_2)
            
            # Remove any duplicate spaces
            duplicateSpacePattern = re.compile(r'\ +')
            conv = re.sub(duplicateSpacePattern, ' ', conv)
            
            indices.append(int(items[0]))
            conversations.append(conv.lower())
    
    if mode == "train":
        return indices, conversations, labels, u1, u2, u3
    else:
        return indices, conversations, u1, u2, u3

In [80]:
trainIndices, trainTexts, u1_train, u2_train, u3_train = preprocessData('data/devwithoutlabels.txt', mode="test")

In [81]:
print(u2_train)

['your a guy <allcaps> not <allcaps> as <allcaps> if <allcaps> you <allcaps> would <allcaps> understand <allcaps>', 'the thing you do.', "and i'm happi for you <smile>", 'left it there oop', 'i said soon master.', 'hey at least i age well!', "what if i told you i'm not?", 'whi tomorrow?', "you say- you'r leav soon . <repeat> anywher you wanna go befor you head?", 'i just did it .l.', 'thank you x‑d', 'done for the day ?', "if you just finish the game . <repeat> then you haven't finish the game . <repeat>", 'whi sorri ! <sadface>', 'depend on how long your internet has been out ! <repeat>', 'thank you. <smile>', 'yeah inde <smile>', 'yeaa i hope soo ! <repeat>', "now you'r tempt me to.", 'oh i am', "i'm the normal one, if you want", 'then', 'video or text', 'whi what', 'but why?', "yeah . <repeat> i don't live in the citi . <repeat> x", 'you never sleep', 'danc to enjoy, not to please.', 'yeah do you have ani plan for today?', 'you did saturday with me but i see how it is', 'whi what ha

In [68]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
print(stemmer.stem("i'm"))

i'm
